In [18]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = {
    'UserID': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'Movie': ['The Matrix', 'Titanic', 'Inception', 'The Matrix', 'Titanic', 'The Godfather', 'Inception', 'The Godfather', 'The Dark Knight'],
    'Rating': [5, 4, 5, 4, 5, 5, 4, 5, 5]
}
df = pd.DataFrame(data)

user_item_matrix = df.pivot_table(index='UserID', columns='Movie', values='Rating')

user_item_matrix_filled = user_item_matrix.fillna(0)
item_similarity = cosine_similarity(user_item_matrix_filled.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

def get_user_ratings():
    print("Please rate the following movies (1-5). If you haven't seen the movie, leave it blank.")
    movies = user_item_matrix.columns.tolist()
    user_ratings = {}

    for movie in movies:
        try:
            rating = input(f"{movie}: ")
            if rating != "":
                user_ratings[movie] = float(rating)
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 5.")

    return pd.Series(user_ratings)

def recommend_movies_from_input(user_ratings, num_recommendations=2):
    similar_movies = pd.Series(dtype=float)
    
    for movie, rating in user_ratings.items():
        similar_movies = pd.concat([similar_movies, item_similarity_df[movie]])
        
    similar_movies = similar_movies.groupby(similar_movies.index).mean()
    recommended_movies = similar_movies.sort_values(ascending=False).drop(user_ratings.index)
    
    return recommended_movies.head(num_recommendations)

if __name__ == "__main__":
    user_ratings = get_user_ratings()
    if user_ratings.empty:
        print("No ratings provided. Unable to make recommendations.")
    else:
        recommendations = recommend_movies_from_input(user_ratings)
        print("\nRecommended Movies for You:\n", recommendations)


Please rate the following movies (1-5). If you haven't seen the movie, leave it blank.


Inception:  
The Dark Knight:  
The Godfather:  
The Matrix:  
Titanic:  


No ratings provided. Unable to make recommendations.
